Retrieve Datapack's data
========================

Repovizz2 can be used to retrieve datapacks with multimodal data, previously [uploaded](Upload.ipynb) by yourself or other users.

We will use here the [repovizz2 client library](Authentication.ipynb) to ease the authentication, but it is basically a standard OAuth2 authentication.

In [1]:
from repovizz2 import RepoVizzClient

We will be using those modules:

In [2]:
import json
# This is for the notebook
from IPython.display import IFrame, display

Now, some global parameters to connect to repovizz2

In [3]:
CLIENT_ID = "4991097c-9a0c-4a56-a337-2b2177c5aa74"
CLIENT_SECRET = "97c055e7-5dd1-49e8-a6da-126d58f07ce1"

Repovizz2 authentication
-----------------------

We authenticate the client (this script) against repoVizz2 using a registered user. For more details on the authentication system see [repoVizz2 authentication](Authentication.ipynb).


This will create an iframe in the notebook to authenticate the client, but you can open a new browser to do so, if you want

In [4]:
repovizz_client = RepoVizzClient(client_id=CLIENT_ID,client_secret=CLIENT_SECRET, repovizz_url="http://localhost:8080")
authorization_url = repovizz_client.start_auth()
repovizz_client.start_server(async=True)

# print authorization_url # and click on it
# - or -
# import webbrowser
# webbrowser.open(authorization_url)
# - or -
display(IFrame(authorization_url,800,600))

In [5]:
repovizz_client.finish_auth()

Getting User basic information
------------------------------

We first get info about our identity. We can use /api/v1.0/user to get json information about the current logged user.

In [6]:
myself = repovizz_client.get("/api/v1.0/user")
username = myself['username']
userid = myself['id']

Listing my datapacks
--------------------

In [7]:
print myself['datapacks']

[u'a4e6b685-afb8-4b40-9c61-6b7687db4723']


Let's check the datapacks names using the `/api/v1.0/datapacks` url

In [8]:
for dp in myself['datapacks']:
    print repovizz_client.get('/api/v1.0/datapacks/{}'.format(dp))['name']

MYO_various


Retrieve a Datapack
-------------------

In [11]:
datapack_id = myself['datapacks'][0] #datapack id to retrieve
datapack = repovizz_client.get('/api/v1.0/datapacks/{}'.format(datapack_id))

This is the datapack structure. The first layer has metadata (owner, name, permissions...) and a member (`structure`) that is the datapack document.

In [12]:
datapack

{u'folder': None,
 u'group_edit_permissions': [],
 u'id': u'a4e6b685-afb8-4b40-9c61-6b7687db4723',
 u'name': u'MYO_various',
 u'owner': 1,
 u'structure': {u'children': [{u'class': u'data',
    u'link': u'repovizz:9ec009d2ea38b6d600f793424739dcdc75bf8a7c',
    u'meta': {u'mime': u'video/mp4; charset=binary'},
    u'name': u'Video',
    u'text': u'Handheld camera',
    u'type': u'e44ab9c4-a1b5-4474-b806-ece64555ef2b'},
   {u'class': u'data',
    u'link': u'repovizz:a910ba389ca2847b6bf548fadad68ccebeee4f54',
    u'meta': {u'mime': u'audio/x-wav; charset=binary'},
    u'name': u'Audio',
    u'text': u'Camera microphone',
    u'type': u'48e3b5d2-c34c-459c-9a2d-0756c20d80de'},
   {u'children': [{u'class': u'data',
      u'link': u'repovizz:0162912010aa5cad7f6d16bb487b4300c7e76942',
      u'name': u'EMG',
      u'text': u'8-channel EMG data from the MYO armband',
      u'type': u'0746bb48-9b92-4cd3-8d9e-3e1de13269eb'}],
    u'class': u'container',
    u'name': u'EMG',
    u'text': u''},
   {u

The leaves of the document have links to the data files. If the link starts with `repovizz:` it means that the file is hosted on repovizz2.

In [13]:
data_id = datapack['structure']['children'][2]['children'][0]['link']
print data_id

repovizz:0162912010aa5cad7f6d16bb487b4300c7e76942


To query the file, remove the `repovizz:` prefix and fetch it with the `/api/v1.0/datapacks/{id}/content/{data_id}` url.

In [14]:
#retrieve data
data_id = data_id.split(':')[1] #remove prefix
d = repovizz_client.get('/api/v1.0/datapacks/{}/content/{}'.format(datapack['id'],data_id),raw=True)

In [18]:
#this file is a json
print d.text[:200]

[
  [-1,1,0,-1,-1,-1,1,1  ],
  [-2,-3,-1,-3,0,-2,-3,-2  ],
  [-2,-3,-1,-3,0,-2,-3,-2  ],
  [-1,-1,-1,-1,1,0,-1,-2  ],
  [-1,-1,-1,0,0,0,-1,-1  ],
  [-1,-1,-1,0,0,0,-1,-1  ],
  [-1,-1,-1,0,0,0,-1,-1  ]


Continue with [Datapack Edition](Edition.ipynb)